<a href="https://colab.research.google.com/github/michaelcassetti1/DATA301_Lab6/blob/main/DATA_301_Lab_6B_Michael_Cassetti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "bay-wheels").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [2]:
key = "AtCmHbfe9XO_ScCslUKsJ1ZiKmF7D_SF35LQzD6an2KUPOPBizzo5MNtCp_BaHR9"
url = "http://dev.virtualearth.net/REST/v1/Locations/US/{adminDistrict}/{postalCode}/{locality}/{addressLine}?includeNeighborhood={includeNeighborhood}&include={includeValue}&maxResults={maxResults}&key={BingMapsKey}"

import pandas as pd
df_stations = pd.read_csv("stations.csv")
df_bay_wheels = df_stations[df_stations['network'] == "bay-wheels"]
df_bay_wheels

,city,network,name,free_bikes,empty_slots,timestamp,longitude,latitude,id,total_bikes
7616,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Harmon St at Adeline St,11,4,2023-02-28T02:03:46.591000Z,-122.270582,37.849735,d0e8f4f1834b7b33a3faf8882f567ab8,15
7617,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Fountain Alley at S 2nd St,4,6,2023-02-28T02:03:46.254000Z,-121.889277,37.336188,983514094dd808b1604da2dcfc2d09af,10
7618,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Oak St at 1st St,15,8,2023-02-28T02:03:46.578000Z,-121.881090,37.322125,da17603652106fda93da4e255a5b0a22,23
7619,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Bestor Art Park,4,10,2023-02-28T02:03:46.381000Z,-121.874119,37.323678,7a21c92b3b4cd2f7759107b4fdebf869,14
7620,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,5th St at Virginia St,4,22,2023-02-28T02:03:46.522000Z,-121.877120,37.325998,ce34d38fb230a23c1ced12d1e16df294,26
...,...,...,...,...,...,...,...,...,...,...
8117,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,36th Ave at Kirkham St,2,17,2023-02-28T02:03:46.374000Z,-122.495129,37.758900,c3288c5dc8fec444070c4aeb739f23e8,19
8118,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Irving St at 17th Ave,4,23,2023-02-28T02:03:46.321000Z,-122.475220,37.763732,1ad00322846e6c78df48cbffbd48a204,27
8119,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Irving St at 18th Ave,0,0,2023-02-28T02:03:46.464000Z,-122.476326,37.763497,9f949acb8f12727a77a3e04ecb066809,0
8120,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,La Playa St at Lincoln Way,22,1,2023-02-28T02:03:46.269000Z,-122.509526,37.763874,ba68e968feece64c5ae1449160642bd1,23


In [3]:
print(df_bay_wheels['city'].value_counts()[0], "total bike stations in SF")

506 total bike stations in SF


## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [4]:
df_bay_wheels

,city,network,name,free_bikes,empty_slots,timestamp,longitude,latitude,id,total_bikes
7616,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Harmon St at Adeline St,11,4,2023-02-28T02:03:46.591000Z,-122.270582,37.849735,d0e8f4f1834b7b33a3faf8882f567ab8,15
7617,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Fountain Alley at S 2nd St,4,6,2023-02-28T02:03:46.254000Z,-121.889277,37.336188,983514094dd808b1604da2dcfc2d09af,10
7618,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Oak St at 1st St,15,8,2023-02-28T02:03:46.578000Z,-121.881090,37.322125,da17603652106fda93da4e255a5b0a22,23
7619,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Bestor Art Park,4,10,2023-02-28T02:03:46.381000Z,-121.874119,37.323678,7a21c92b3b4cd2f7759107b4fdebf869,14
7620,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,5th St at Virginia St,4,22,2023-02-28T02:03:46.522000Z,-121.877120,37.325998,ce34d38fb230a23c1ced12d1e16df294,26
...,...,...,...,...,...,...,...,...,...,...
8117,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,36th Ave at Kirkham St,2,17,2023-02-28T02:03:46.374000Z,-122.495129,37.758900,c3288c5dc8fec444070c4aeb739f23e8,19
8118,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Irving St at 17th Ave,4,23,2023-02-28T02:03:46.321000Z,-122.475220,37.763732,1ad00322846e6c78df48cbffbd48a204,27
8119,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,Irving St at 18th Ave,0,0,2023-02-28T02:03:46.464000Z,-122.476326,37.763497,9f949acb8f12727a77a3e04ecb066809,0
8120,"('San Francisco Bay Area, CA', 'bay-wheels')",bay-wheels,La Playa St at Lincoln Way,22,1,2023-02-28T02:03:46.269000Z,-122.509526,37.763874,ba68e968feece64c5ae1449160642bd1,23


## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.